In [2]:
import pysam
import pandas as pd
import isabl_cli as ii
from pathlib import Path

outdir = "/work/isabl/home/zhouy1/tmp/svclone_test"
sample = "IID_H158186_T01_01_WG01"
experiment = ii.Experiment(sample)
bam = experiment.bam_files.GRCh37.url
svin = Path(outdir).joinpath(f"{sample}.svin")
#SVclone simple format
#chr1	pos1	chr2	pos2
annotsvs = [x for x in experiment.results if x.application.name == "ANNOT_SVS"][0]
svs = []
with pysam.VariantFile(annotsvs.results.pass_vcf, mode="r") as vcf:
    for rec in vcf:
        svs.append([rec.chrom, rec.pos, rec.info["CHR2"], rec.stop, rec.info["SVTYPE"], rec.info["STRANDS"]])
svs = pd.DataFrame(svs, columns=["chr1","pos1","chr2","pos2","svtype","strand"])
svs.to_csv(svin, sep="\t", index=False)
print(f"python SVclone.py annotate -i {svin} -b {bam} -s {sample} -o {outdir} --sv_format simple -cfg {outdir}/svclone_config.ini")
pa_path = Path(outdir).joinpath(f"{sample}_svin.txt")
postannotate = pd.read_csv(pa_path,sep="\t")
postannotate.merge(svs,left_on=["original_pos1","original_pos2"],right_on=["pos1","pos2"])
print("Unable to annotate:", postannotate[postannotate["dir1"]=="?"].shape[0])
print(f"python SVclone.py count -i {Path(outdir).joinpath(f'{sample}_svin.txt')} -b {bam} -s {sample} -o {outdir} -cfg {outdir}/svclone_config.ini")

python SVclone.py annotate -i /work/isabl/home/zhouy1/tmp/svclone_test/IID_H158186_T01_01_WG01.svin -b /work/isabl/data/analyses/08/79/230879/IID_H158186_T01_01_WG01.bam -s IID_H158186_T01_01_WG01 -o /work/isabl/home/zhouy1/tmp/svclone_test --sv_format simple -cfg /work/isabl/home/zhouy1/tmp/svclone_test/svclone_config.ini
Unable to annotate: 6
python SVclone.py count -i /work/isabl/home/zhouy1/tmp/svclone_test/IID_H158186_T01_01_WG01_svin.txt -b /work/isabl/data/analyses/08/79/230879/IID_H158186_T01_01_WG01.bam -s IID_H158186_T01_01_WG01 -o /work/isabl/home/zhouy1/tmp/svclone_test -cfg /work/isabl/home/zhouy1/tmp/svclone_test/svclone_config.ini
